<a href="https://colab.research.google.com/github/ktshih/ktshih_64061/blob/main/Assignment%202/Shih_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Make copy of data to home directory
!cp /content/drive/MyDrive/Upload/KSUMPH/MIS64061/PYLibrary/dogs-vs-cats.zip /content/.

# Unzip data
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

Mounted at /content/drive


In [4]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)